In [9]:
%load_ext cython

In [ ]:
%%cython

# cython: infer_types=True
# cython: language_level=3
# cython: boundscheck=False
# cython: wraparound=False

import numpy as np
from scipy import linalg as la
from cython.parallel import prange, parallel
from scipy.linalg.cython_lapack cimport zgetrf, zgetrs, zgetri, zlange, zlacpy
from scipy.linalg.cython_blas cimport zgemm, zaxpy

from libc.stdlib cimport abort, malloc, free, rand

cdef double conv = 1e-8


cdef inline complex conj(complex a) nogil:
    return a.real - a.imag*1j


cdef inline void init_vecs(complex[:,:] h_ii, complex[:,:] s_ii,
                           complex[:,:] h_ij, complex[:,:] s_ij,
                           complex* v_00, complex* v_01,
                           complex* v_10, complex* v_11,
                           double energy, double eta=1e-5, double bias=0.) nogil:

    cdef int m = h_ii.shape[0], i, j
    cdef complex z
    z = energy - bias + eta * 1.j

    for j in range(m):#, schedule='static'):
        for i in range(m):
            v_00[j*m+i] = z * conj(s_ii[i,j]) - conj(h_ii[i,j])
            v_11[j*m+i] = v_00[j*m+i]
            v_01[j*m+i] = z * conj(s_ij[i,j]) - conj(h_ij[i,j])

    for j in range(m):#, schedule='static'):
        for i in range(m):
            v_10[j*m+i] = z * s_ij[j,i] - h_ij[j,i]
            
            
def py_init_vecs(h_ii, s_ii, h_ij, s_ij,
            py_v_00, py_v_01, py_v_10, py_v_11,
            energy, eta=1e-5, bias=0.):

    z = energy - bias + eta * 1.j

    py_v_00[:] = z * s_ii.T.conj() - h_ii.T.conj()
    py_v_11[:] = py_v_00.copy()
    py_v_10[:] = z * s_ij - h_ij
    py_v_01[:] = z * s_ij.T.conj() - h_ij.T.conj()


def test_init_vecs():

    m = 3
    nkpts = 2
    a = np.random.random((nkpts,m,m)).astype(complex)

    cdef complex *v_00
    cdef complex *v_01
    cdef complex *v_10
    cdef complex *v_11

    cdef int sz = m*m*sizeof(double)
    v_00 = <complex*>malloc(sz)
    v_01 = <complex*>malloc(sz)
    v_10 = <complex*>malloc(sz)
    v_11 = <complex*>malloc(sz)
    cdef complex[:] v_00_view = <complex[:m*m]> v_00
    cdef complex[:] v_01_view = <complex[:m*m]> v_01
    cdef complex[:] v_10_view = <complex[:m*m]> v_10
    cdef complex[:] v_11_view = <complex[:m*m]> v_11

    py_v_00 = np.zeros((m,m), complex)
    py_v_10 = np.zeros((m,m), complex)
    py_v_01 = np.zeros((m,m), complex)
    py_v_11 = np.zeros((m,m), complex)

    args_common = a[0]*4
    args_py = py_v_00, py_v_01, py_v_10, py_v_11
    args_pyx = v_00, v_01, v_10, v_11

    py_init_vecs(a[0],a[0],a[0],a[0],py_v_00, py_v_01, py_v_10, py_v_11,0.)
    init_vecs(a[0],a[0],a[0],a[0],v_00,v_01,v_10,v_11,0.)      
    
    
    assert np.allclose(np.asarray(v_00_view).reshape(m,m), py_v_00)
    assert False
    free(v_00)
    free(v_01)
    free(v_10)
    free(v_11)
    
test_init_vecs()

In [1]:
%load_ext cython

In [46]:
%%cython

import numpy as np

cdef inline void transpose(complex* a, complex* b, int m) nogil:
    cdef int i, j
    for i in range(m):
        for j in range(m):
            a[i*m+j] = b[j*m+i]
            
a = np.arange(9).reshape((3,3)).astype(complex)
b = np.zeros((2,3,3), complex)
c = np.zeros_like(a)

# a = np.asfortranarray(a)

cdef complex[:,:] _a = a
cdef complex[:,:] _b 
_b = b[0]

transpose(&_b[0,0],&_a[0,0],a.shape[0])

c = b[0]

In [47]:
c

array([[0.+0.j, 3.+0.j, 6.+0.j],
       [1.+0.j, 4.+0.j, 7.+0.j],
       [2.+0.j, 5.+0.j, 8.+0.j]])

In [41]:
%%cython

import numpy as np
cimport numpy as np

cdef int m = 3
cdef inline f(np.ndarray[complex, ndim=1] a):
    cdef int i, j
    for i in range(m):
        for j in range(m):
            a[i*m+j] = b[j*m+i]

cdef complex *_a 
_a = <complex*> a.data
cdef int i, j

for i in range(m):
    for j in range(m):
        _a[i*m+j] *= 2.+1.j    

print(a)


Error compiling Cython file:
------------------------------------------------------------
...

import numpy as np
cimport numpy as np

cdef int m = 3
cdef np.ndarray[complex, ndim=1] a
                                ^
------------------------------------------------------------

/home/gag/.cache/ipython/cython/_cython_magic_25ef3ffe131a6ddd568873c0d3601ffc.pyx:6:33: Buffer types only allowed as function local variables

Error compiling Cython file:
------------------------------------------------------------
...

import numpy as np
^
------------------------------------------------------------

/home/gag/.cache/ipython/cython/_cython_magic_25ef3ffe131a6ddd568873c0d3601ffc.pyx:2:0: Buffer vars not allowed in module scope


In [33]:
a.shape

(3, 3)

In [34]:
a

array([[0.+0.j, 1.+0.j, 2.+0.j],
       [3.+0.j, 4.+0.j, 5.+0.j],
       [6.+0.j, 7.+0.j, 8.+0.j]])

In [7]:
b

array([[[0.+0.j, 1.+0.j, 2.+0.j],
        [3.+0.j, 4.+0.j, 5.+0.j],
        [6.+0.j, 7.+0.j, 8.+0.j]],

       [[0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j]]])